In [1]:
import igl

In [2]:
#https://github.com/libigl/libigl-tutorial-data

In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time

In [22]:
v, f = igl.read_triangle_mesh('data/arm.obj')
VE,EE,_,BE,CE,_ = igl.read_tgf('data/arm.tgf')
W = igl.read_dmat('data/arm-weights.dmat')
labels = np.load('data/hand.label.npy').astype(int)
# v -= v.min(axis=0)
# v /= v.max()

In [106]:
Vb = np.concatenate((v, VE))
Eb = EE + v.shape[0]

In [107]:
labels = np.concatenate((np.zeros(v.shape[0]), np.array(range(1,VE.shape[0]+1)))).astype(int)

In [109]:
handle_vertex_positions = Vb.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = Vb[slices] + np.array([[x,y,z]])
    # v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    # p.update_object(vertices = v_deformed)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [110]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    # widgets_dict = dict()
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [111]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    return target_pos
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

' (Optional) Register this function to perform interactive deformation\npos_f.deformer = position_deformer\n'

In [115]:

## Widget UI

p = mp.plot(Vb, f)
# p = mp.plot(v,f)
# p.add_points(VE)
p.add_edges(VE,EE, shading={"line_color": "green"});
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0001610…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4, 5), value=1), FloatSlider(value=0.0, desc…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

In [116]:
handle_vertex_positions

array([[-0.692193  ,  0.084945  ,  0.0399108 ],
       [-0.693609  ,  0.090783  ,  0.0441324 ],
       [-0.707763  ,  0.0805516 ,  0.038791  ],
       ...,
       [ 0.08140069,  0.18519857, -0.04254464],
       [ 0.5987815 , -0.07760989,  0.06525213],
       [ 0.73572894, -0.17523362,  0.09021015]])